In [1]:
!neofetch

/usr/bin/sh: 1: neofetch: not found


In [1]:
import os
from os.path import join as J
import pandas as pd
import numpy as np
import subprocess

In [2]:
data_path = "~/work/ai-academy2020/data/"

In [3]:
!tree {data_path}

/usr/bin/sh: 1: tree: not found


In [4]:
pd.read_csv( J(data_path, "small_group_description.csv") ).head()

,small_group,small_group_code
0,Зоотовары,0
1,Сетевые супермаркеты и продуктовые магазины,1
2,Обслуживание существующего автомобиля (АЗС),2
3,Оплата телефона и связи,3
4,Аптеки,4


In [6]:
transactions_train = pd.read_csv( J(data_path, "transactions_train.csv") )

In [7]:
transactions_train.head(2)

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017


In [33]:
transactions_train["trans_date"]

0             6
1             6
2             8
3             9
4            10
           ... 
26450572    727
26450573    727
26450574    727
26450575    727
26450576    729
Name: trans_date, Length: 26450577, dtype: int64

In [44]:
def prep_date(data):
    data["trans_month"] = ((data["trans_date"] // 30) % 12 + 1)
    data["trans_year"] = ((data["trans_date"] // 365) + 1)

In [46]:
prep_date(transactions_train)
transactions_train.trans_year.value_counts()

2    14682465
1    11768112
Name: trans_year, dtype: int64

* client_id - уникальный идентификатор клиента
* trans_date - дата совершения транзакции
* small_group - категория покупки
* amount_rur - сумма транзакции

In [49]:
transactions_train.head(1)

,client_id,trans_date,small_group,amount_rur,trans_month,trans_year
0,33172,6,4,71.463,1,1


In [55]:
def get_first_last_transaction_date(data):
    ret = data.groupby('client_id')["trans_date"].agg(["min", "max"])
    ret.columns = ["first_transaction_date", "last_transaction_date"]
    return ret

In [58]:
def get_minmax_yearly_trans(data):
    ret = data.groupby(['client_id','trans_year'])["amount_rur"].agg(['count', 'sum'])\
        .groupby(['client_id'])[['count', 'sum']].agg(['max', 'min']).reset_index()

    ret.columns = ['client_id', 'yeartrans_max', 'yeartrans_min', 'yearcount_max', 'yearcount_min']
    ret["yearcount_diff"] = ret["yearcount_max"] - ret["yearcount_min"]
    return ret
get_minmax_yearly_trans(transactions_train)

,client_id,yeartrans_max,yeartrans_min,yearcount_max,yearcount_min,yearcount_diff
0,4,363,357,15264.437,13139.684,2124.753
1,6,373,357,7978.615,7742.124,236.491
2,7,482,291,27803.883,25826.153,1977.730
3,10,411,295,20585.312,13834.053,6751.259
4,11,450,362,18019.155,8770.249,9248.906
...,...,...,...,...,...,...
29995,49993,515,386,14364.269,9883.275,4480.994
29996,49995,533,436,15331.092,12620.064,2711.028
29997,49996,602,536,44188.509,36711.836,7476.673
29998,49997,488,227,8125.031,5168.084,2956.947


In [61]:
def get_minmax_monthly_trans(data):
    ret = data.groupby(['client_id','trans_month'])["amount_rur"].agg(['count', 'sum'])\
        .groupby(['client_id'])[['count', 'sum']].agg(['max', 'min']).reset_index()

    ret.columns = ['client_id', 'monthtrans_max', 'monthtrans_min', 'monthcount_max', 'monthcount_min']
    ret["monthcount_diff"] = ret["monthcount_max"] - ret["monthcount_min"]
    return ret
get_minmax_monthly_trans(transactions_train)

,client_id,monthtrans_max,monthtrans_min,monthcount_max,monthcount_min,monthcount_diff
0,4,66,52,3636.275,1658.041,1978.234
1,6,82,42,2178.090,863.566,1314.524
2,7,74,28,10053.767,1666.168,8387.599
3,10,88,42,5169.318,1513.850,3655.468
4,11,91,56,3966.888,936.898,3029.990
...,...,...,...,...,...,...
29995,49993,94,58,3013.794,1495.437,1518.357
29996,49995,123,62,4270.710,1342.261,2928.449
29997,49996,132,62,12893.703,2107.605,10786.098
29998,49997,91,21,1630.802,524.260,1106.542


In [8]:
def get_row_with_max_col_val(df, col, group_col):
    idx = df.groupby(group_col)[col].transform(max) == df[col]
    return df[idx]

In [9]:
agg_features=transactions_train.groupby('client_id')['amount_rur'].agg(['sum','mean','std','min','max']).reset_index()

In [10]:
max_group = get_row_with_max_col_val(transactions_train, 'amount_rur', 'client_id')[ ["client_id", "small_group"] ]

In [11]:
agg_features.head()

,client_id,sum,mean,std,min,max
0,4,28404.121,39.450168,73.511624,0.043,1341.802
1,6,15720.739,21.535259,26.200397,0.045,315.781
2,7,53630.036,69.379089,253.261383,0.043,4505.971
3,10,34419.365,48.752642,63.191701,0.045,654.893
4,11,26789.404,32.991877,107.395139,0.388,2105.058


In [12]:
counter_df_train=transactions_train.groupby(['client_id','small_group'])['amount_rur'].count()

In [13]:
sum_df_train = transactions_train.groupby(['client_id','small_group'])['amount_rur'].agg(["sum","max"])

In [14]:
sum_df_train.head(1)

,,sum,max
client_id,small_group,,
4,1,15049.897,265.798


# Посмотрим на максимальные - минимальные затраты за 1 день

In [32]:
transactions_train

,client_id,trans_date,small_group,amount_rur
0,33172,6,4,71.463
1,33172,6,35,45.017
2,33172,8,11,13.887
3,33172,9,11,15.983
4,33172,10,11,21.341
...,...,...,...,...
26450572,43300,727,25,7.602
26450573,43300,727,15,3.709
26450574,43300,727,1,6.448
26450575,43300,727,11,24.669


In [15]:
transactions_train.groupby(['client_id','trans_date'])["amount_rur"].agg(['count', 'sum']).agg(['max', 'min'])

,count,sum
max,14,413371.598
min,1,0.001


In [16]:
minmax_dayly_transactions_train = \
transactions_train.groupby(['client_id','trans_date'])["amount_rur"].agg(['count', 'sum'])\
.groupby(['client_id'])[['count', 'sum']].agg(['max', 'min']).reset_index()

minmax_dayly_transactions_train.columns = ['client_id', 'daytrans_max', 'daytrans_min', 'daycount_max', 'daycount_min']

In [17]:
cat_counts_train=counter_df_train.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values='amount_rur')
cat_counts_train=cat_counts_train.fillna(0)
cat_counts_train.columns=['sg_count_'+str(i) for i in cat_counts_train.columns]

In [18]:
cat_sum_train = sum_df_train.reset_index().pivot(index='client_id', \
                                                      columns='small_group',values=["sum","max"])
cat_sum_train = cat_sum_train.fillna(0)
cat_sum_train.columns=['sg_sum_'+str(i) for i in cat_sum_train.columns]

In [19]:
cat_sum_train.head()

,"sg_sum_('sum', 0)","sg_sum_('sum', 1)","sg_sum_('sum', 2)","sg_sum_('sum', 3)","sg_sum_('sum', 4)","sg_sum_('sum', 5)","sg_sum_('sum', 6)","sg_sum_('sum', 7)","sg_sum_('sum', 8)","sg_sum_('sum', 9)",...,"sg_sum_('max', 192)","sg_sum_('max', 193)","sg_sum_('max', 195)","sg_sum_('max', 196)","sg_sum_('max', 197)","sg_sum_('max', 198)","sg_sum_('max', 199)","sg_sum_('max', 200)","sg_sum_('max', 202)","sg_sum_('max', 203)"
client_id,,,,,,,,,,,,,,,,,,,,,
4,0.000,15049.897,98.966,835.403,3799.572,0.000,0.0,0.000,22.852,514.861,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,53.372,9602.770,0.000,2905.083,189.480,0.000,0.0,0.000,0.000,80.193,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,89.072,4632.372,125.521,888.523,660.002,1241.557,0.0,118.975,197.820,3568.778,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10,639.798,13440.506,24.928,2034.764,2422.679,0.000,0.0,0.000,270.330,722.931,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
11,88.239,5037.288,0.000,637.526,178.423,1844.468,0.0,0.000,0.000,86.990,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
minmax_dayly_transactions_train

,client_id,daytrans_max,daytrans_min,daycount_max,daycount_min
0,4,5,1,1341.802,1.282
1,6,4,1,412.967,2.290
2,7,7,1,4534.593,0.065
3,10,7,1,792.126,1.221
4,11,4,1,2112.019,0.458
...,...,...,...,...,...
29995,49993,5,1,1325.683,1.897
29996,49995,7,1,1243.601,0.690
29997,49996,8,1,2245.405,4.270
29998,49997,5,1,863.182,1.509


In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score


def experiment():
    y_train = pd.read_csv( J(data_path, "train_target.csv") )
    train=pd.merge(y_train,cat_counts_train,on='client_id')
    

In [78]:
train

,client_id,sum,mean,std,min,max,sg_count_0,sg_count_1,sg_count_2,sg_count_3,...,monthcount_max,monthcount_min,monthcount_diff,yeartrans_max,yeartrans_min,yearcount_max,yearcount_min,yearcount_diff,first_transaction_date,last_transaction_date
0,24662,30254.011,34.774725,72.037354,0.074,1227.314,0.0,174.0,2.0,64.0,...,5299.353,1335.528,3963.825,444,426,15295.465,14958.546,336.919,0,695
1,1046,42548.570,52.015367,106.540962,0.550,1210.506,1.0,187.0,61.0,47.0,...,5311.449,2241.328,3070.121,481,337,22440.602,20107.968,2332.634,0,729
2,34089,26842.816,34.325852,59.927450,0.043,782.641,0.0,372.0,0.0,72.0,...,3749.132,1431.592,2317.540,420,362,18470.716,8372.100,10098.616,4,729
3,34848,15773.126,16.160990,14.224936,0.043,109.590,0.0,359.0,1.0,0.0,...,2023.154,720.864,1302.290,538,438,7994.095,7779.031,215.064,10,729
4,47076,12488.375,15.929050,35.473591,0.432,541.165,0.0,378.0,0.0,150.0,...,1998.422,351.017,1647.405,399,385,7436.374,5052.001,2384.373,1,728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30106,14303,285001.426,345.456274,1009.576868,1.786,9919.347,0.0,284.0,26.0,31.0,...,48914.807,12417.936,36496.871,468,357,155499.239,129502.187,25997.052,0,729
30107,22301,34950.321,49.715962,116.602464,0.043,2136.877,2.0,168.0,9.0,64.0,...,4142.795,1823.246,2319.549,400,303,21392.851,13557.470,7835.381,0,729
30108,25731,54074.454,49.338005,80.490725,0.181,884.970,0.0,266.0,8.0,5.0,...,8651.979,2023.383,6628.596,655,441,33673.864,20400.590,13273.274,73,729
30109,16820,32494.435,42.643615,136.489534,0.388,2959.186,2.0,340.0,3.0,79.0,...,7181.225,1221.195,5960.030,448,314,16554.899,15939.536,615.363,10,729


In [27]:
y_train

,client_id,bins
0,24662,2
1,1046,0
2,34089,2
3,34848,1
4,47076,3
...,...,...
29995,14303,1
29996,22301,2
29997,25731,0
29998,16820,3


In [28]:
agg_features

,client_id,sum,mean,std,min,max
0,4,28404.121,39.450168,73.511624,0.043,1341.802
1,6,15720.739,21.535259,26.200397,0.045,315.781
2,7,53630.036,69.379089,253.261383,0.043,4505.971
3,10,34419.365,48.752642,63.191701,0.045,654.893
4,11,26789.404,32.991877,107.395139,0.388,2105.058
...,...,...,...,...,...,...
29995,49993,24247.544,26.911814,73.592787,0.211,1315.470
29996,49995,27951.156,28.845362,64.723186,0.690,1243.601
29997,49996,80900.345,71.089934,125.642727,0.458,1657.546
29998,49997,13293.115,18.591769,38.841011,0.432,858.240


In [31]:
minmax_dayly_transactions_train

,client_id,daytrans_max,daytrans_min,daycount_max,daycount_min
0,4,5,1,1341.802,1.282
1,6,4,1,412.967,2.290
2,7,7,1,4534.593,0.065
3,10,7,1,792.126,1.221
4,11,4,1,2112.019,0.458
...,...,...,...,...,...
29995,49993,5,1,1325.683,1.897
29996,49995,7,1,1243.601,0.690
29997,49996,8,1,2245.405,4.270
29998,49997,5,1,863.182,1.509


In [63]:
y_train = pd.read_csv( J(data_path, "train_target.csv") )
train=pd.merge(y_train,agg_features,on='client_id')
train=pd.merge(train,cat_counts_train.reset_index(),on='client_id')
train=pd.merge(train,cat_sum_train.reset_index(),on='client_id')

train=pd.merge(train,max_group.reset_index()[["client_id", "small_group"]],on='client_id')
train=pd.merge(train,minmax_dayly_transactions_train,on='client_id')

In [64]:
train=pd.merge(train,get_minmax_monthly_trans(transactions_train),on='client_id')
train=pd.merge(train,get_minmax_yearly_trans(transactions_train),on='client_id')

In [68]:
train=pd.merge(train,get_first_last_transaction_date(transactions_train).reset_index(),on='client_id')

In [69]:
train

,client_id,bins,sum,mean,std,min,max,sg_count_0,sg_count_1,sg_count_2,...,monthcount_max,monthcount_min,monthcount_diff,yeartrans_max,yeartrans_min,yearcount_max,yearcount_min,yearcount_diff,first_transaction_date,last_transaction_date
0,24662,2,30254.011,34.774725,72.037354,0.074,1227.314,0.0,174.0,2.0,...,5299.353,1335.528,3963.825,444,426,15295.465,14958.546,336.919,0,695
1,1046,0,42548.570,52.015367,106.540962,0.550,1210.506,1.0,187.0,61.0,...,5311.449,2241.328,3070.121,481,337,22440.602,20107.968,2332.634,0,729
2,34089,2,26842.816,34.325852,59.927450,0.043,782.641,0.0,372.0,0.0,...,3749.132,1431.592,2317.540,420,362,18470.716,8372.100,10098.616,4,729
3,34848,1,15773.126,16.160990,14.224936,0.043,109.590,0.0,359.0,1.0,...,2023.154,720.864,1302.290,538,438,7994.095,7779.031,215.064,10,729
4,47076,3,12488.375,15.929050,35.473591,0.432,541.165,0.0,378.0,0.0,...,1998.422,351.017,1647.405,399,385,7436.374,5052.001,2384.373,1,728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30106,14303,1,285001.426,345.456274,1009.576868,1.786,9919.347,0.0,284.0,26.0,...,48914.807,12417.936,36496.871,468,357,155499.239,129502.187,25997.052,0,729
30107,22301,2,34950.321,49.715962,116.602464,0.043,2136.877,2.0,168.0,9.0,...,4142.795,1823.246,2319.549,400,303,21392.851,13557.470,7835.381,0,729
30108,25731,0,54074.454,49.338005,80.490725,0.181,884.970,0.0,266.0,8.0,...,8651.979,2023.383,6628.596,655,441,33673.864,20400.590,13273.274,73,729
30109,16820,3,32494.435,42.643615,136.489534,0.388,2959.186,2.0,340.0,3.0,...,7181.225,1221.195,5960.030,448,314,16554.899,15939.536,615.363,10,729


In [71]:
train[:1].shape

(1, 630)

In [72]:
y_train = train["bins"]
del train["bins"]

In [74]:
import catboost as cb
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
# import xgboost as xgb

In [79]:
train

,client_id,sum,mean,std,min,max,sg_count_0,sg_count_1,sg_count_2,sg_count_3,...,monthcount_max,monthcount_min,monthcount_diff,yeartrans_max,yeartrans_min,yearcount_max,yearcount_min,yearcount_diff,first_transaction_date,last_transaction_date
0,24662,30254.011,34.774725,72.037354,0.074,1227.314,0.0,174.0,2.0,64.0,...,5299.353,1335.528,3963.825,444,426,15295.465,14958.546,336.919,0,695
1,1046,42548.570,52.015367,106.540962,0.550,1210.506,1.0,187.0,61.0,47.0,...,5311.449,2241.328,3070.121,481,337,22440.602,20107.968,2332.634,0,729
2,34089,26842.816,34.325852,59.927450,0.043,782.641,0.0,372.0,0.0,72.0,...,3749.132,1431.592,2317.540,420,362,18470.716,8372.100,10098.616,4,729
3,34848,15773.126,16.160990,14.224936,0.043,109.590,0.0,359.0,1.0,0.0,...,2023.154,720.864,1302.290,538,438,7994.095,7779.031,215.064,10,729
4,47076,12488.375,15.929050,35.473591,0.432,541.165,0.0,378.0,0.0,150.0,...,1998.422,351.017,1647.405,399,385,7436.374,5052.001,2384.373,1,728
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30106,14303,285001.426,345.456274,1009.576868,1.786,9919.347,0.0,284.0,26.0,31.0,...,48914.807,12417.936,36496.871,468,357,155499.239,129502.187,25997.052,0,729
30107,22301,34950.321,49.715962,116.602464,0.043,2136.877,2.0,168.0,9.0,64.0,...,4142.795,1823.246,2319.549,400,303,21392.851,13557.470,7835.381,0,729
30108,25731,54074.454,49.338005,80.490725,0.181,884.970,0.0,266.0,8.0,5.0,...,8651.979,2023.383,6628.596,655,441,33673.864,20400.590,13273.274,73,729
30109,16820,32494.435,42.643615,136.489534,0.388,2959.186,2.0,340.0,3.0,79.0,...,7181.225,1221.195,5960.030,448,314,16554.899,15939.536,615.363,10,729


In [76]:
model = cb.CatBoostClassifier(iterations = 5000, learning_rate = 0.04, depth = 5,
                              loss_function = 'MultiClass', eval_metric='Accuracy',
                              random_state = 53, task_type="GPU", devices='0', verbose = 1)

res = cross_validate(model, train, y_train, cv=3, verbose=1)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


0:	learn: 0.5053806	total: 10.8ms	remaining: 53.8s
1:	learn: 0.5162914	total: 19.8ms	remaining: 49.4s
2:	learn: 0.5263551	total: 29ms	remaining: 48.3s
3:	learn: 0.5260064	total: 38ms	remaining: 47.5s
4:	learn: 0.5287963	total: 47.7ms	remaining: 47.6s
5:	learn: 0.5372658	total: 56.7ms	remaining: 47.2s
6:	learn: 0.5410522	total: 66ms	remaining: 47.1s
7:	learn: 0.5436927	total: 75.5ms	remaining: 47.1s
8:	learn: 0.5444898	total: 85ms	remaining: 47.1s
9:	learn: 0.5444400	total: 94.1ms	remaining: 47s
10:	learn: 0.5481267	total: 104ms	remaining: 46.9s
11:	learn: 0.5505181	total: 113ms	remaining: 47.1s
12:	learn: 0.5499203	total: 122ms	remaining: 46.9s
13:	learn: 0.5505181	total: 131ms	remaining: 46.8s
14:	learn: 0.5528597	total: 141ms	remaining: 46.9s
15:	learn: 0.5549522	total: 150ms	remaining: 46.9s
16:	learn: 0.5558489	total: 160ms	remaining: 46.9s
17:	learn: 0.5560980	total: 169ms	remaining: 46.8s
18:	learn: 0.5567457	total: 178ms	remaining: 46.8s
19:	learn: 0.5580411	total: 187ms	remaini

164:	learn: 0.6271921	total: 1.43s	remaining: 41.9s
165:	learn: 0.6271423	total: 1.44s	remaining: 41.9s
166:	learn: 0.6277401	total: 1.45s	remaining: 41.9s
167:	learn: 0.6279394	total: 1.46s	remaining: 41.9s
168:	learn: 0.6283380	total: 1.46s	remaining: 41.9s
169:	learn: 0.6290355	total: 1.47s	remaining: 41.8s
170:	learn: 0.6289857	total: 1.48s	remaining: 41.7s
171:	learn: 0.6288860	total: 1.49s	remaining: 41.7s
172:	learn: 0.6292846	total: 1.49s	remaining: 41.6s
173:	learn: 0.6289857	total: 1.5s	remaining: 41.6s
174:	learn: 0.6296333	total: 1.51s	remaining: 41.6s
175:	learn: 0.6298326	total: 1.51s	remaining: 41.5s
176:	learn: 0.6302312	total: 1.52s	remaining: 41.5s
177:	learn: 0.6309785	total: 1.53s	remaining: 41.4s
178:	learn: 0.6311778	total: 1.54s	remaining: 41.4s
179:	learn: 0.6312774	total: 1.54s	remaining: 41.4s
180:	learn: 0.6310283	total: 1.55s	remaining: 41.3s
181:	learn: 0.6315265	total: 1.56s	remaining: 41.3s
182:	learn: 0.6317756	total: 1.57s	remaining: 41.2s
183:	learn: 0

349:	learn: 0.6620167	total: 2.83s	remaining: 37.7s
350:	learn: 0.6621662	total: 2.84s	remaining: 37.6s
351:	learn: 0.6624651	total: 2.85s	remaining: 37.6s
352:	learn: 0.6625149	total: 2.85s	remaining: 37.6s
353:	learn: 0.6623655	total: 2.86s	remaining: 37.6s
354:	learn: 0.6623655	total: 2.87s	remaining: 37.6s
355:	learn: 0.6625149	total: 2.88s	remaining: 37.5s
356:	learn: 0.6627142	total: 2.88s	remaining: 37.5s
357:	learn: 0.6634117	total: 2.89s	remaining: 37.5s
358:	learn: 0.6632124	total: 2.9s	remaining: 37.5s
359:	learn: 0.6635612	total: 2.91s	remaining: 37.5s
360:	learn: 0.6635612	total: 2.91s	remaining: 37.4s
361:	learn: 0.6638103	total: 2.92s	remaining: 37.4s
362:	learn: 0.6635612	total: 2.93s	remaining: 37.4s
363:	learn: 0.6641092	total: 2.93s	remaining: 37.4s
364:	learn: 0.6639099	total: 2.94s	remaining: 37.4s
365:	learn: 0.6643085	total: 2.95s	remaining: 37.3s
366:	learn: 0.6646572	total: 2.96s	remaining: 37.3s
367:	learn: 0.6647071	total: 2.96s	remaining: 37.3s
368:	learn: 0

521:	learn: 0.6819450	total: 4.04s	remaining: 34.7s
522:	learn: 0.6821443	total: 4.05s	remaining: 34.7s
523:	learn: 0.6826923	total: 4.06s	remaining: 34.7s
524:	learn: 0.6822937	total: 4.07s	remaining: 34.7s
525:	learn: 0.6825927	total: 4.08s	remaining: 34.7s
526:	learn: 0.6823436	total: 4.08s	remaining: 34.7s
527:	learn: 0.6819450	total: 4.09s	remaining: 34.6s
528:	learn: 0.6822439	total: 4.1s	remaining: 34.6s
529:	learn: 0.6819948	total: 4.1s	remaining: 34.6s
530:	learn: 0.6821941	total: 4.11s	remaining: 34.6s
531:	learn: 0.6827421	total: 4.12s	remaining: 34.6s
532:	learn: 0.6826923	total: 4.12s	remaining: 34.6s
533:	learn: 0.6827919	total: 4.13s	remaining: 34.5s
534:	learn: 0.6830909	total: 4.14s	remaining: 34.5s
535:	learn: 0.6830411	total: 4.14s	remaining: 34.5s
536:	learn: 0.6831905	total: 4.15s	remaining: 34.5s
537:	learn: 0.6831905	total: 4.16s	remaining: 34.5s
538:	learn: 0.6832902	total: 4.17s	remaining: 34.5s
539:	learn: 0.6830909	total: 4.17s	remaining: 34.5s
540:	learn: 0.

...

4415:	learn: 0.8556983	total: 29.6s	remaining: 3.92s
4416:	learn: 0.8558976	total: 29.6s	remaining: 3.91s
4417:	learn: 0.8558478	total: 29.6s	remaining: 3.9s
4418:	learn: 0.8558478	total: 29.6s	remaining: 3.9s
4419:	learn: 0.8558478	total: 29.6s	remaining: 3.89s
4420:	learn: 0.8557980	total: 29.7s	remaining: 3.88s
4421:	learn: 0.8556485	total: 29.7s	remaining: 3.88s
4422:	learn: 0.8554991	total: 29.7s	remaining: 3.87s
4423:	learn: 0.8557980	total: 29.7s	remaining: 3.86s
4424:	learn: 0.8557980	total: 29.7s	remaining: 3.86s
4425:	learn: 0.8557980	total: 29.7s	remaining: 3.85s
4426:	learn: 0.8557980	total: 29.7s	remaining: 3.84s
4427:	learn: 0.8557980	total: 29.7s	remaining: 3.84s
4428:	learn: 0.8557980	total: 29.7s	remaining: 3.83s
4429:	learn: 0.8556983	total: 29.7s	remaining: 3.82s
4430:	learn: 0.8558478	total: 29.7s	remaining: 3.82s
4431:	learn: 0.8557482	total: 29.7s	remaining: 3.81s
4432:	learn: 0.8558478	total: 29.7s	remaining: 3.8s
4433:	learn: 0.8558478	total: 29.7s	remaining: 3.

4599:	learn: 0.8606794	total: 30.8s	remaining: 2.68s
4600:	learn: 0.8605300	total: 30.8s	remaining: 2.67s
4601:	learn: 0.8605300	total: 30.8s	remaining: 2.67s
4602:	learn: 0.8605300	total: 30.8s	remaining: 2.66s
4603:	learn: 0.8605798	total: 30.8s	remaining: 2.65s
4604:	learn: 0.8605798	total: 30.9s	remaining: 2.65s
4605:	learn: 0.8606794	total: 30.9s	remaining: 2.64s
4606:	learn: 0.8606794	total: 30.9s	remaining: 2.63s
4607:	learn: 0.8606794	total: 30.9s	remaining: 2.63s
4608:	learn: 0.8609285	total: 30.9s	remaining: 2.62s
4609:	learn: 0.8609285	total: 30.9s	remaining: 2.61s
4610:	learn: 0.8610281	total: 30.9s	remaining: 2.6s
4611:	learn: 0.8610281	total: 30.9s	remaining: 2.6s
4612:	learn: 0.8611277	total: 30.9s	remaining: 2.59s
4613:	learn: 0.8610779	total: 30.9s	remaining: 2.58s
4614:	learn: 0.8610281	total: 30.9s	remaining: 2.58s
4615:	learn: 0.8610779	total: 30.9s	remaining: 2.57s
4616:	learn: 0.8611775	total: 30.9s	remaining: 2.56s
4617:	learn: 0.8612273	total: 30.9s	remaining: 2

4786:	learn: 0.8653616	total: 32s	remaining: 1.42s
4787:	learn: 0.8653118	total: 32s	remaining: 1.42s
4788:	learn: 0.8654612	total: 32s	remaining: 1.41s
4789:	learn: 0.8654114	total: 32s	remaining: 1.4s
4790:	learn: 0.8652122	total: 32s	remaining: 1.4s
4791:	learn: 0.8654114	total: 32.1s	remaining: 1.39s
4792:	learn: 0.8653616	total: 32.1s	remaining: 1.38s
4793:	learn: 0.8654612	total: 32.1s	remaining: 1.38s
4794:	learn: 0.8655111	total: 32.1s	remaining: 1.37s
4795:	learn: 0.8655111	total: 32.1s	remaining: 1.36s
4796:	learn: 0.8654612	total: 32.1s	remaining: 1.36s
4797:	learn: 0.8655609	total: 32.1s	remaining: 1.35s
4798:	learn: 0.8657103	total: 32.1s	remaining: 1.34s
4799:	learn: 0.8656605	total: 32.1s	remaining: 1.34s
4800:	learn: 0.8656107	total: 32.1s	remaining: 1.33s
4801:	learn: 0.8656107	total: 32.1s	remaining: 1.32s
4802:	learn: 0.8656605	total: 32.1s	remaining: 1.32s
4803:	learn: 0.8658597	total: 32.1s	remaining: 1.31s
4804:	learn: 0.8657601	total: 32.1s	remaining: 1.3s
4805:	

4943:	learn: 0.8696453	total: 33s	remaining: 374ms
4944:	learn: 0.8696453	total: 33s	remaining: 367ms
4945:	learn: 0.8697450	total: 33s	remaining: 361ms
4946:	learn: 0.8698446	total: 33s	remaining: 354ms
4947:	learn: 0.8698446	total: 33.1s	remaining: 347ms
4948:	learn: 0.8700936	total: 33.1s	remaining: 341ms
4949:	learn: 0.8699442	total: 33.1s	remaining: 334ms
4950:	learn: 0.8699442	total: 33.1s	remaining: 327ms
4951:	learn: 0.8699442	total: 33.1s	remaining: 321ms
4952:	learn: 0.8699442	total: 33.1s	remaining: 314ms
4953:	learn: 0.8699442	total: 33.1s	remaining: 307ms
4954:	learn: 0.8699442	total: 33.1s	remaining: 301ms
4955:	learn: 0.8699442	total: 33.1s	remaining: 294ms
4956:	learn: 0.8700936	total: 33.1s	remaining: 287ms
4957:	learn: 0.8701933	total: 33.1s	remaining: 281ms
4958:	learn: 0.8702431	total: 33.1s	remaining: 274ms
4959:	learn: 0.8701435	total: 33.1s	remaining: 267ms
4960:	learn: 0.8700936	total: 33.1s	remaining: 261ms
4961:	learn: 0.8701933	total: 33.1s	remaining: 254ms
4

[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:  1.8min finished


In [80]:
np.mean(res['test_score'])

0.6379400505662773

In [81]:
param={'objective':'multi:softprob','num_class':4,'n_jobs':16,'seed':42}
model_xgb = xgb.XGBClassifier(**param,n_estimators=600)

NameError: name 'xgb' is not defined

In [109]:
cross_val_score(
    model_xgb,
    train,
    y_train,
    cv = 4,
    scoring = "accuracy",
    verbose = 1,
)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:  3.6min finished


array([0.62350598, 0.625     , 0.62189451, 0.63114536])

In [110]:
np.mean([0.62350598, 0.625     , 0.62189451, 0.63114536])

0.6253864625

In [86]:
np.mean([0.62124834, 0.62207758, 0.62269164, 0.6290194 ])

0.62375924

In [57]:
np.mean([0.61341301, 0.6186238 , 0.61525176, 0.61506777])

0.615589085

In [32]:
np.mean([0.61580345, 0.61517003, 0.6124618 , 0.61639649])

0.6149579425

In [31]:
np.mean([0.61383631, 0.61813333, 0.61154821, 0.61594879])

0.61486666

# Знаю как реализовать
* - Максимальная транзакция
* - КАтегория максимальной транзакции
* Количество транзакций ниже трешхолда
* Трешхолд для каждого пользователя свой (типо выше среднего)
* Уровень трат выше среднего уровня трат для каждой категории
* Максимальные затраты за 1 день
* Сколько в среднем тратит за 1 день
* Производная от графика (затраты / дни)

# Надо думать

* как считается trans_date ? (Относительно получения карты или абсолютно)
* если абсолютно, то можно попытаться определить дни недели